In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
from os import path

from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from IPython.display import SVG
import re

In [116]:
pairs = pd.read_csv('data/Main_RPAIRS_KEGG.tsv', sep='\t')

pairs['source'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[0])
pairs['target'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[1])

# Drop columns that won't be used
pairs.drop(['KEGG_reactions', 'CAR', 'RPAIR_main'], axis=1, inplace=True)
pairs.shape

(10747, 3)

In [117]:
df = pd.read_csv('data/compounds_list_KEGG.csv')
display(df.head())

print(df.shape)

,id,mol_weight,formula
0,C00002,507.1810,C10H16N5O13P3
1,C00003,664.4330,C21H28N7O14P2
2,C00005,745.4209,C21H30N7O17P3
3,C00007,31.9988,O2
4,C00011,44.0095,CO2


(5620, 3)


How many nans

In [118]:
df.isna().sum()

id              0
mol_weight    680
formula         2
dtype: int64

In [119]:
# These are probably mistakes
display(df[df['formula'].isna()])

,id,mol_weight,formula
665,C15778,NaN,NaN
2845,C20798,NaN,NaN


### Fix manually these mistakes

**Correct *C15778***

In [120]:
display(pairs[pairs['source'] == 'C15778'])

display(pairs[pairs['target'] == 'C15778'])

,Reactant_pair,source,target
787,C15778_C15781,C15778,C15781


,Reactant_pair,source,target


According to **KEGG** for the Reaction *R07492*, *C15778* should be *C15780*

In [121]:
pairs.loc[pairs.index[787], 'Reactant_pair'] = 'C15780_C15781'

df.loc[df.index[665], 'id'] = 'C15780'
df.loc[df.index[665], 'mol_weight'] = 396.6484
df.loc[df.index[665], 'formula'] = 'C28H44O'

**Correct *C20798***

In [122]:
display(pairs[pairs['source'] == 'C20798'])

display(pairs[pairs['target'] == 'C20798'])

,Reactant_pair,source,target
5112,C20798_C21180,C20798,C21180
5767,C20798_C20831,C20798,C20831


,Reactant_pair,source,target
890,C19675_C20798,C19675,C20798
2787,C11499_C20798,C11499,C20798


According to **KEGG** for the Reaction *R10719*, *C20798* should be *C21181*

In [123]:
pairs.loc[pairs.index[5112], 'Reactant_pair'] = 'C21181_C21180'
pairs.loc[pairs.index[5767], 'Reactant_pair'] = 'C21181_C20831'
pairs.loc[pairs.index[890], 'Reactant_pair'] = 'C19675_C21181'
pairs.loc[pairs.index[2787], 'Reactant_pair'] = 'C11499_C21181'

df.loc[df.index[2845], 'id'] = 'C21181'
df.loc[df.index[2845], 'mol_weight'] = 154.1417
df.loc[df.index[2845], 'formula'] = 'C3H6O5S'

Recalculate *source* and *target* in pairs

In [124]:
pairs['source'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[0])
pairs['target'] = pairs['Reactant_pair'].apply(lambda x: x.split('_')[1])

pairs.to_csv('data/Main_RPAIRS_KEGG_fixed.csv', index=None)

## Exctract Features From Chemical Formula

In [125]:
def extract_elements(df, column_name):
    # define the regular expression pattern to match the chemical formula
    pattern = r'[A-Z][a-z]?'
    # initialize a set to store the element symbols
    elements = set()
    # loop over the values in the specified column of the DataFrame
    for value in df[column_name].values:
        # find all matches of the pattern in the value string
        matches = re.findall(pattern, value)
        # add the matches to the set of elements
        elements.update(matches)
    return elements

def extract_stoichiometry(formula):
    # define the regular expression pattern to match the chemical formula
    pattern = r'([A-Z][a-z]?)(\d*)'
    # initialize the dictionary to store the element symbol and its stoichiometry
    stoichiometry = {}
    # loop over the matches of the pattern in the formula string
    for match in re.findall(pattern, formula):
        symbol, count = match
        # if the count is empty, set it to 1
        count = int(count) if count else 1
        # add the symbol and count to the stoichiometry dictionary
        stoichiometry[symbol] = count
    return stoichiometry

# example usage
elements = extract_elements(df, 'formula')
print(elements)

# Create a col for every element
for elm in elements: df[elm]=0

{'Zn', 'X', 'As', 'N', 'C', 'P', 'S', 'Cl', 'Se', 'H', 'Co', 'F', 'I', 'Br', 'Mo', 'R', 'O', 'Mg'}


In [126]:
for row in range(len(df)):
    
    formula = df['formula'].iloc[row]
    stoichiometry = extract_stoichiometry(formula)
    
    for key, value in stoichiometry.items():
        #df[key].iloc[row] = value
        df.loc[df.index[row], key] = value

display(df.head(3))

,id,mol_weight,formula,Zn,X,As,N,C,P,S,...,Se,H,Co,F,I,Br,Mo,R,O,Mg
0,C00002,507.1810,C10H16N5O13P3,0,0,0,5,10,3,0,...,0,16,0,0,0,0,0,0,13,0
1,C00003,664.4330,C21H28N7O14P2,0,0,0,7,21,2,0,...,0,28,0,0,0,0,0,0,14,0
2,C00005,745.4209,C21H30N7O17P3,0,0,0,7,21,3,0,...,0,30,0,0,0,0,0,0,17,0


In [127]:
# Many many compounds with R
print(len(df[df['R']!=0]))

# Col that contains the info if the compound is a polymer or not
df['polymer'] = 0

for row in range(len(df)):
    if 'n' in df['formula'].iloc[row]: 
        df.loc[df.index[row], 'polymer'] = 1
        
# Print how many polymers do we have
print(len(df[df['polymer'] == 1]))

# save to csv
df.to_csv('data/curated.csv')

597
129


**Here we re-fill all mol_weight based on paper NICEpath methodology.**

# Load curated dataset from Stef - Filled NaN `mol_weight`

In [128]:
df = pd.read_csv('data/curated_complete_mw.csv', index_col=0)
df.drop('mol_weight', axis=1, inplace=True)
df.drop('R.1', axis=1, inplace=True)
df.drop('Mr', axis=1, inplace=True)
df.rename({'N.1':'mol_weight'}, axis=1, inplace=True)
df.head()

,id,formula,Co,C,Br,Zn,N,Mo,S,Se,...,X,H,O,As,F,R,P,Cl,polymer,mol_weight
0,C00002,C10H16N5O13P3,0,10,0,0,5,0,0,0,...,0,16,13,0,0,0,3,0,0,507.0
1,C00003,C21H28N7O14P2,0,21,0,0,7,0,0,0,...,0,28,14,0,0,0,2,0,0,664.0
2,C00005,C21H30N7O17P3,0,21,0,0,7,0,0,0,...,0,30,17,0,0,0,3,0,0,745.0
3,C00007,O2,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,32.0
4,C00011,CO2,0,1,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,44.0


In [129]:
source_mw = df.set_index('id')['mol_weight']
target_mw = source_mw.reindex(pairs['target']).values

pairs['MW'] = abs(source_mw.reindex(pairs['source']).values - target_mw) / (source_mw.reindex(pairs['source']).values + target_mw)

display(pairs.head(3))

,Reactant_pair,source,target,MW
0,C00002_C07024,C00002,C07024,0.001976
1,C00003_C00004,C00003,C00004,0.000752
2,C00005_C00006,C00005,C00006,0.000672


# Import to Networkx 

In [130]:
import networkx as nx

G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 5620 
# edges: 10747


In [131]:
if path.exists('data/nodes_centralities.csv'):
    dc = pd.read_csv('data/nodes_centralities.csv', index_col=0)
else:
    # PageRank
    pr = nx.pagerank(G)
    pr = pd.DataFrame(list(pr.items()), columns=['Node', 'PageRank'])

    # degree centrality
    dc = nx.degree_centrality(G)
    dc = pd.DataFrame(list(dc.items()), columns=['Node', 'Degree Centrality'])
    
    # Betweenness centrality
    bc = nx.betweenness_centrality(G)
    bv = pd.DataFrame(list(bc.items()), columns=['Node', 'Betweenness Centrality'])
    
    # centralities
    dc['PageRank'] = pr['PageRank'].copy()
    dc['Betweenness Centrality'] = bc['Betweenness Centrality'].copy()
    dc.sort_values(by='PageRank', ascending=False, inplace=True)
    dc = pd.merge(dc, df[['id','formula','mol_weight']].rename({'id':'Node'}, axis=1), on='Node')
    dc.to_csv('data/nodes_centralities.csv')

#### Scale the centrality measures

In [132]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dc[['Degree Centrality', 'PageRank', 'Betweenness Centrality']] = scaler.fit_transform(dc[['Degree Centrality', 'PageRank', 'Betweenness Centrality']])

print('Centrality measures statistics: ')
display(dc.describe().T)

Centrality measures statistics: 


,count,mean,std,min,25%,50%,75%,max
Degree Centrality,5620.0,0.003087,0.019770,0.0000,1.092896e-03,0.001093,0.003279,1.0000
PageRank,5620.0,0.003621,0.020670,0.0000,1.266360e-03,0.002221,0.003468,1.0000
Betweenness Centrality,5620.0,0.001034,0.016605,0.0000,1.114510e-07,0.000079,0.000719,1.0000
mol_weight,5620.0,440.817497,328.063873,17.0305,1.881794e+02,320.466300,589.540650,2484.8138


In [133]:
# Number of compounds with centrality measure over the 75% threshold
print(len(dc[dc['Betweenness Centrality'] > 0.000719]))

print(len(dc[dc['Degree Centrality'] > 0.0034680]))

print(len(dc[dc['PageRank'] > 0.003279]))

1055
785
1572


In [134]:
# top_centrality_nodes = list(dc[dc['Degree Centrality'] > 0.0034680]['Node'].values)

# subset_df = df[df['id'].isin(top_centrality_nodes)]
# subset_df

In [135]:
# Concat to df the centrality measures from dc
df = pd.merge(df, dc[['Node', 'PageRank', 'Degree Centrality', 'Betweenness Centrality']], left_on='id', right_on='Node')
df.drop('Node', axis=1, inplace=True)
df.head(2)

,id,formula,Co,C,Br,Zn,N,Mo,S,Se,...,As,F,R,P,Cl,polymer,mol_weight,PageRank,Degree Centrality,Betweenness Centrality
0,C00002,C10H16N5O13P3,0,10,0,0,5,0,0,0,...,0,0,0,3,0,0,507.0,0.220897,0.244809,0.147827
1,C00003,C21H28N7O14P2,0,21,0,0,7,0,0,0,...,0,0,0,2,0,0,664.0,0.007615,0.009836,0.000216


In [136]:
# Create new mw adding info from centrality measure
df['mw'] = df['mol_weight'] + df['mol_weight'] * df['PageRank']

# Update the edge weight
source_mw = df.set_index('id')['mw']
target_mw = source_mw.reindex(pairs['target']).values

pairs['MW_wCentr'] = abs(source_mw.reindex(pairs['source']).values - target_mw) / (source_mw.reindex(pairs['source']).values + target_mw)

display(pairs.head(3))

,Reactant_pair,source,target,MW,MW_wCentr
0,C00002_C07024,C00002,C07024,0.001976,0.100431
1,C00003_C00004,C00003,C00004,0.000752,0.001602
2,C00005_C00006,C00005,C00006,0.000672,0.000305


# Node2Vec

In [137]:
from karateclub import Node2Vec

# Map node names to integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)

" Perform node embedding using Node2Vec "
N2vec_model = Node2Vec(walk_number=3, walk_length=20,q=.25,\
                       dimensions=124)
N2vec_model.fit(G)
N2Vec_embedding = N2vec_model.get_embedding()
print('Embedding array shape (nodes x features):',N2Vec_embedding.shape )

Embedding array shape (nodes x features): (5620, 124)


In [138]:
# Add a new column to data.x with the node names from the mapping
node_names = [None] * len(N2Vec_embedding)

for node, index in mapping.items():
    node_names[index] = node   
n2v_data = np.column_stack((N2Vec_embedding, node_names))
n2v_data = pd.DataFrame(n2v_data)

In [139]:
n2v_data

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,-1.0449119,-0.3422351,2.1226737,-0.825907,-0.6866515,0.31758073,-0.7017876,0.6705272,0.57523173,2.3003216,...,0.57071024,0.018226353,-0.9647105,1.5617219,0.28642473,0.47198075,0.9911752,0.06922167,1.1137574,C00002
1,-0.3270226,-0.07741775,-0.3220685,0.03158295,-0.11116919,0.16761468,-0.4296445,0.07667206,-0.24127343,0.18206055,...,-0.4394623,0.45566213,-0.39797544,0.25232366,0.28190807,-0.018293912,-0.3403866,0.57648855,-0.363073,C07024
2,-0.5210517,-0.11448687,0.5951467,-0.5011175,0.088545494,0.3869262,-0.011103804,0.05946386,0.07395431,0.21845849,...,0.501521,-0.18770397,-0.22488359,0.346571,0.23060352,-0.18720125,0.17227599,-0.5088462,-0.18688953,C00003
3,-0.21862468,0.027389532,0.43877676,0.10360384,0.5238174,0.14362124,-0.12855531,0.37323418,-0.03449264,0.26456717,...,-0.047759067,0.42599148,0.059778806,0.0023812335,-0.07561561,-0.2395751,-0.081682235,-0.10532175,0.005744969,C00004
4,-0.2859471,-0.041897338,-0.032474544,0.12630127,0.040036667,0.26506242,-0.25880778,0.10990681,-0.1594407,0.18235837,...,-0.030489655,0.2777166,-0.07548805,0.113553636,0.042652886,0.040060297,-0.11106613,0.16816995,-0.22937354,C00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5615,0.18610471,-0.212359,-0.044793155,-0.22825393,0.014153947,-0.023685439,0.42217517,-0.1323149,-0.012421756,0.32507527,...,0.009476825,-0.13422634,-0.9616381,0.117700085,0.19513033,-0.13279262,1.2247697,-0.07271011,-0.44091016,C16487
5616,0.4275823,0.047002833,0.523819,-3.0697203e-05,0.3359713,-0.22322875,0.22157124,-0.4678844,-0.27466068,0.14823005,...,-0.045367923,0.33653504,-0.22347227,-1.2389548,-0.01727226,-0.4344419,0.6110761,-0.7512497,-0.34317058,C00237
5617,-0.09166668,-0.06784172,-0.03328557,-0.06709327,0.06372226,-0.04259375,0.058141343,-0.1325647,-0.037182465,-0.022515478,...,0.056181394,-0.11548245,-0.09877825,-0.13095433,-0.26248187,-0.07324306,0.21608512,-0.08632513,-0.16003616,C05232
5618,0.22474119,0.11850264,0.26618195,-0.45715696,-0.17687611,0.15604168,-0.28208834,-0.056057222,-0.35716617,-0.10043573,...,-0.020736374,0.08714113,-0.38049817,0.1968601,0.55577356,-0.29544693,0.11800518,-0.25311255,-0.11354024,C00283


In [153]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = n2v_data.iloc[23,:-1]
v_embedding = n2v_data.iloc[24,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), \
                               v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

0.4034862765732707


In [142]:
n2v_data[n2v_data[124] == 'C00031']

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
23,0.0334939,-0.24580842,0.37516624,-0.36860693,0.15893765,-0.2736735,0.34171018,-0.14455976,0.33901134,-0.33756584,...,-0.021715797,-0.26396927,-1.0833231,-0.037483007,-0.32397985,0.3238737,0.15301631,-0.843441,-0.214997,C00031


In [147]:
n2v_data[n2v_data[124] == 'C00103']

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124


## Add edge attributes

In [100]:
def get_weights(a,b, method):
    
    # a_C and b_C is the number of C at each compound
    '''
    Checking if we go from a compound with C to a compound with no C
    '''
    a_C = df[df['id']==a]['C'].values
    b_C = df[df['id']==b]['C'].values
    
    if (a_C != 0 and b_C == 0) or (a_C == 0 and b_C != 0):
        return 999
    
    try:
        w = pairs[(pairs['source'] == a) & (pairs['target'] == b)][method].values[0]
    except IndexError:
        w = pairs[(pairs['source'] == b) & (pairs['target'] == a)][method].values[0]
    
    return w

for edge in G.edges():
    G.edges[(edge[0], edge[1])]['weight'] = \
            get_weights(edge[0], edge[1], method='MW')

In [106]:
source = 'C00082'
target = 'C01533'
list(nx.shortest_path(G, source, target, weight='weight'))

['C00082',
 'C00811',
 'C01197',
 'C01494',
 'C05619',
 'C00482',
 'C05610',
 'C02325',
 'C01533']

### Load file with test cases fron NICEpath

In [28]:
test_cases = pd.read_csv('data/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases.head(3)

,Pathway,source,target
0,"C00082,C00811,C01197,C01494,C05619,C00482,C056...",C00082,C01533
1,"C00223,C00323,C00406,C02666,C00590,C00761",C00223,C00761
2,"C00811,C01197,C01494,C05619,C00482,C01175,C02887",C00811,C02887


In [29]:
for row in range(len(test_cases)):
    source = test_cases['source'].iloc[row]
    target = test_cases['target'].iloc[row]
    print(list(nx.shortest_path(G, source, target, weight='weight')))

['C00082', 'C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00223', 'C12096', 'C00029', 'C00761']
['C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C01175', 'C02887']
['C00079', 'C00166', 'C05607', 'C16257', 'C00540', 'C00903']
['C00223', 'C12096', 'C00029', 'C17750']
['C06561', 'C00509', 'C16492', 'C01460']
['C05903', 'C12249', 'C00029', 'C12635', 'C12636']
['C01477', 'C01460', 'C00029', 'C00167', 'C04900']
['C05903', 'C00389', 'C10107', 'C12633', 'C11620']
['C05905', 'C08604', 'C08639', 'C12096', 'C16299']
['C05904', 'C12137', 'C12642', 'C00083', 'C12647']
['C05908', 'C12138', 'C00029', 'C16303']
['C00078', 'C00398', 'C21762', 'C21778', 'C21779', 'C07576']
['C06160', 'C06161', 'C00019', 'C21591', 'C20299']
['C05191', 'C05247', 'C05194', 'C05193', 'C05316', 'C06165']
['C00079', 'C00166', 'C05607', 'C16257', 'C10860']
['C00047', 'C04076', 'C00450', 'C00408']
['C07481', 'C07130', 'C16357', 'C00385']
['C00082', 'C00355', 'C03758', 'C08557']
['C08539', 

### Get node embeddings using GNN and Clustering (idea)

In [147]:
# Backup df
df_copy = df.copy()

In [148]:
from sklearn.preprocessing import StandardScaler

'''Scale df'''
# Select the columns with non-object data types
numeric_cols = df.select_dtypes(include=['int', 'float']).columns

# Apply standard scaling to the selected columns
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

'''Recreate Graph'''
G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 5620 
# edges: 10747


In [150]:
from torch_geometric.utils.convert import from_networkx
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv
import torch
import torch.nn as nn
import torch.nn.functional as F

node_features = [col for col in df.drop(['id', 'formula'], axis=1).columns]

data = from_networkx(G, group_node_attrs=node_features)
print(data.x.shape)

# Map node names to integers
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)

# Add a new column to data.x with the node names from the mapping
node_names = [None] * len(data.x)
for node, index in mapping.items():
    node_names[index] = node
    
graph_data = np.column_stack((data.x, node_names))
graph_data = pd.DataFrame(graph_data)

torch.Size([5620, 24])


In [184]:
'''
Graph embedding good method (maybe)
'''
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GAE

# Define the model architecture
class Encoder(torch.nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 4)
        self.conv2 = GCNConv(4, 8)
        self.conv3 = GCNConv(8, 4)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        X = F.relu(x)
        x = self.conv3(x, edge_index)
        return x

class Decoder(torch.nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.linear = torch.nn.Linear(4, data.num_node_features)

    def forward(self, z):
        x = self.linear(z)
        return torch.sigmoid(x)

model = GAE(Encoder(), Decoder())

# Define the loss function and the optimizer
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the model
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z)
    loss = criterion(out, data.x)
    loss.backward()
    optimizer.step()
    
    if(epoch % 20 == 0):
        print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss.item()))
    
    
model.eval()
emb = model.encode(data.x, data.edge_index)
print(emb.shape)

Epoch: 000, Loss: 0.7349
Epoch: 020, Loss: 0.2246
Epoch: 040, Loss: -2.6157
Epoch: 060, Loss: -7.2248
Epoch: 080, Loss: -8.5579
torch.Size([5620, 4])


In [185]:
node_embeddings=pd.DataFrame(emb.detach().numpy())
node_embeddings['id']  = graph_data[graph_data.shape[1]-1]
# node_embeddings.set_index('id', inplace=True)
node_embeddings

,0,1,2,3,id
0,93.795853,143.531647,35.195347,-107.759872,C00002
1,16.805361,20.991673,8.238287,-14.824412,C07024
2,22.670057,31.883059,9.498896,-23.496105,C00003
3,32.198071,28.360352,22.004126,-16.508657,C00004
4,34.567509,27.109596,25.170515,-14.389196,C00005
...,...,...,...,...,...
5615,6.346945,8.030086,2.494330,-4.765063,C16487
5616,-29.497427,28.562559,-47.055561,-39.296860,C00237
5617,105.250984,50.594093,88.269920,-7.195076,C05232
5618,-12.522794,11.806552,-20.213654,-15.982663,C00283


In [186]:
display(node_embeddings[node_embeddings['id'] == 'C00031'])
display(node_embeddings[node_embeddings['id'] == 'C00002'])

,0,1,2,3,id
23,-3.992759,26.801254,-16.969198,-28.848948,C00031


,0,1,2,3,id
0,93.795853,143.531647,35.195347,-107.759872,C00002


In [188]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = node_embeddings.iloc[23,:-1]
v_embedding = node_embeddings.iloc[5619,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

0.9373832828161275


In [147]:
class SAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers):
        super(SAGE, self).__init__()
        self.num_layers = num_layers
        self.convs = nn.ModuleList()
        
        for i in range(num_layers):
            in_channels = in_channels if i == 0 else hidden_channels
            self.convs.append(SAGEConv(in_channels, hidden_channels))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x

    def full_forward(self, x, edge_index):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != self.num_layers - 1:
                x = x.relu()
                x = F.dropout(x, p=0.5, training=self.training)
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SAGE(data.num_node_features, hidden_channels=4, \
             num_layers=1)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x, edge_index = data.x.to(device), data.edge_index.to(device)

# Forward propagate the graph through the model
model.eval()
with torch.no_grad():
    out = model.full_forward(data.x, data.edge_index)

# Extract the node embeddings
node_embeddings = out.numpy()  # or out.detach().numpy() if using PyTorch version <1.6

# Print the shape of the node embeddings matrix
print('Node embeddings shape:', node_embeddings.shape)

Node embeddings shape: (5620, 4)


In [148]:
node_embeddings=pd.DataFrame(node_embeddings)
node_embeddings['id']  = graph_data[graph_data.shape[1]-1]
# node_embeddings.set_index('id', inplace=True)
node_embeddings

,0,1,2,3,id
0,0.441722,-0.347565,-2.931345,0.594922,C00002
1,0.597105,1.616578,1.081657,-0.932573,C07024
2,0.461805,1.199976,0.614995,-0.415841,C00003
3,0.393747,1.137706,0.687152,-0.301356,C00004
4,0.593800,1.507571,1.299703,-0.936374,C00005
...,...,...,...,...,...
5615,4.777917,3.426768,-0.974567,-0.767836,C16487
5616,-0.086657,0.286581,0.003130,0.550068,C00237
5617,0.838591,-0.145284,1.091916,-1.103059,C05232
5618,0.010520,0.095059,-0.517729,0.396282,C00283


In [149]:
node_embeddings[node_embeddings['id'] == 'C00811']

,0,1,2,3,id
2836,0.120062,0.357436,0.464212,-0.107965,C00811


In [150]:
node_embeddings[node_embeddings['id'] == 'C00013']

,0,1,2,3,id
4147,0.034261,-0.132567,-1.27845,0.256991,C00013


In [151]:
node_embeddings[node_embeddings['id'] == 'C00146']

,0,1,2,3,id
3809,-0.006893,0.218506,0.591352,-0.033146,C00146


In [158]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# Suppose you have the node embeddings in a dictionary called 'embeddings'
u_embedding = node_embeddings.iloc[62,:-1]
v_embedding = node_embeddings.iloc[3809,:-1]

similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
print(similarity)

0.4756511205004293


In [153]:
similarity, _ = pearsonr(u_embedding, v_embedding)
print(similarity)

0.38470953227899024


In [154]:
distance = np.linalg.norm(u_embedding - v_embedding)
similarity = 1 / (1 + distance)
print(similarity)

0.24253645835351192


In [159]:
# Add cosine similarity from embedding to pairs 
cos_sim = []
for row in range(len(pairs)):
    
    source = pairs['source'].iloc[row]
    target = pairs['target'].iloc[row]
    
    u_embedding = node_embeddings[node_embeddings['id'] == source].iloc[0, :-1]
    v_embedding = node_embeddings[node_embeddings['id'] == target].iloc[0, :-1]

    similarity = cosine_similarity(u_embedding.values.reshape(1, -1), v_embedding.values.reshape(1, -1))[0, 0]
    cos_sim.append(similarity)
    
pairs['cos_sim'] = cos_sim
# pairs['cos_sim'] = (pairs['cos_sim'] + 1) / 2
pairs['cos_sim'] = 1 - pairs['cos_sim']

pairs.head()

,Reactant_pair,source,target,MW,MW_wCentr,cos_sim
0,C00002_C07024,C00002,C07024,0.001976,0.100431,1.590680
1,C00003_C00004,C00003,C00004,0.000752,0.001602,0.005330
2,C00005_C00006,C00005,C00006,0.000672,0.000305,0.004183
3,C00007_C00027,C00007,C00027,0.030303,0.280698,1.898238
4,C00011_C00058,C00011,C00058,0.022222,0.053514,0.437329


In [162]:
print(pairs['cos_sim'].describe())

pairs['cos_sim'] = pairs['cos_sim'].apply(lambda x: max(0,x))

count    1.074700e+04
mean     9.511019e-01
std      7.280763e-01
min     -2.220446e-16
25%      1.909888e-01
50%      9.043138e-01
75%      1.726002e+00
max      1.998921e+00
Name: cos_sim, dtype: float64


# Import to Networkx 

In [163]:
import networkx as nx

df = df_copy.copy()
G = nx.Graph()

# iterate over the rows of the DataFrame and add each edge to the graph
for index, row in pairs.iterrows():
    source = row['source']
    target = row['target']
    G.add_edge(source, target)

# Add df cols as node features
node_data = df.set_index('id').to_dict('index')

# Add the node features to the graph
for node, data in G.nodes(data=True):
    node_features = node_data.get(node)
    if node_features:
        data.update(node_features)

print('# nodes:', G.number_of_nodes(), "\n# edges:", G.number_of_edges())

# nodes: 5620 
# edges: 10747


## Add edge attributes

In [164]:
def get_weights(a,b, method):
    
    # a_C and b_C is the number of C at each compound
    a_C = df[df['id']==a]['C'].values
    b_C = df[df['id']==b]['C'].values
    if (a_C != 0 and b_C == 0) or (a_C == 0 and b_C != 0):
        return 999
    
    try:
        w = pairs[(pairs['source'] == a) & (pairs['target'] == b)][method].values[0]
    except IndexError:
        w = pairs[(pairs['source'] == b) & (pairs['target'] == a)][method].values[0]
    
    return w

for edge in G.edges():
    G.edges[(edge[0], edge[1])]['weight'] = \
            get_weights(edge[0], edge[1], method='cos_sim')

In [165]:
source = 'C00082'
target = 'C01533'
list(nx.shortest_path(G, source, target, weight='weight'))

['C00082',
 'C00811',
 'C01197',
 'C01494',
 'C05619',
 'C00482',
 'C05610',
 'C02325',
 'C01533']

In [166]:
source = 'C07481'
target = 'C00385'
list(nx.shortest_path(G, source, target, weight='weight'))

['C07481', 'C13747', 'C00067', 'C16353', 'C00385']

### Load file with test cases fron NICEpath

In [167]:
test_cases = pd.read_csv('data/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases.head(3)

,Pathway,source,target
0,"C00082,C00811,C01197,C01494,C05619,C00482,C056...",C00082,C01533
1,"C00223,C00323,C00406,C02666,C00590,C00761",C00223,C00761
2,"C00811,C01197,C01494,C05619,C00482,C01175,C02887",C00811,C02887


In [168]:
for row in range(len(test_cases)):
    source = test_cases['source'].iloc[row]
    target = test_cases['target'].iloc[row]
    print(list(nx.shortest_path(G, source, target, weight='weight')))

['C00082', 'C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00223', 'C00323', 'C00406', 'C12203', 'C05619', 'C01494', 'C02666', 'C00590', 'C00761']
['C00811', 'C01197', 'C01494', 'C05619', 'C00482', 'C01175', 'C02887']
['C00079', 'C00082', 'C00811', 'C01197', 'C01494', 'C05619', 'C12203', 'C00406', 'C00323', 'C00223', 'C00540', 'C00903']
['C00223', 'C00323', 'C00406', 'C17741', 'C10443', 'C17749', 'C17750']
['C06561', 'C00509', 'C16492', 'C01460']
['C05903', 'C12249', 'C12634', 'C12635', 'C12636']
['C01477', 'C04608', 'C04858', 'C01623', 'C00015', 'C00167', 'C04900']
['C05903', 'C00389', 'C10107', 'C12633', 'C11620']
['C05905', 'C08604', 'C08639', 'C12096', 'C16299']
['C05904', 'C12137', 'C08725', 'C12640', 'C12641', 'C12647']
['C05908', 'C12138', 'C16301', 'C16304', 'C16303']
['C00078', 'C00398', 'C21762', 'C21778', 'C21779', 'C07576']
['C06160', 'C03758', 'C00355', 'C00082', 'C02993', 'C00062', 'C00179', 'C00134', 'C00041', 'C00037', 'C00025', 'C0001